In [ ]:
!pip -q install torch torchvision torchaudio transformers datasets scikit-learn


In [ ]:
import pandas as pd

DATA_FILE = "D:\\Code\\skripsi_sentimen\\02.Preprocessing\\data\\preprocessed_A_stepwise_without_normalisasi.csv"
df = pd.read_csv(DATA_FILE)

print("Distribusi Label:")
print(df["label"].value_counts())
df.head(10)


Distribusi Label:
label
NETRAL     721
NEGATIF    585
POSITIF    128
Name: count, dtype: int64


,id_str,created_at,user_id_str,conversation_id_str,full_text,label,text_casefold,text_clean,text_normalized,tokens,tokens_nostop,tokens_stem,text_stemmed,emoji_drop_from_raw,emoji_map_from_raw,emoji_drop_from_stemmed,emoji_map_from_stemmed
0,1608764220451213313,Fri Dec 30 09:57:34 +0000 2022,230609460,1608764220451213313,udah beli tiket di aplikasi kaiaccess dan saat...,NEGATIF,udah beli tiket di aplikasi kaiaccess dan saat...,udah beli tiket di aplikasi kaiaccess dan saat...,sudah beli tiket di aplikasi kaiaccess dan saa...,"['sudah', 'beli', 'tiket', 'di', 'aplikasi', '...","['beli', 'tiket', 'aplikasi', 'pembelian', 'pe...","['beli', 'tiket', 'aplikasi', 'beli', 'bayar',...",beli tiket aplikasi beli bayar mengikutsertaka...,udah beli tiket di aplikasi kaiaccess dan saat...,udah beli tiket di aplikasi kaiaccess dan saat...,beli tiket aplikasi beli bayar mengikutsertaka...,beli tiket aplikasi beli bayar mengikutsertaka...
1,1608631716184489986,Fri Dec 30 01:11:02 +0000 2022,1158718283870138368,1608631716184489986,@KAI121 kenapa gabisa pake ovo buat pembayaran...,NEGATIF,@kai121 kenapa gabisa pake ovo buat pembayaran...,kenapa gabisa pake ovo buat pembayaran kaiacce...,kenapa enggak bisa pakai ovo buat pembayaran k...,"['kenapa', 'enggak', 'bisa', 'pakai', 'ovo', '...","['enggak', 'pakai', 'ovo', 'buat', 'pembayaran']","['enggak', 'pakai', 'ovo', 'buat', 'bayar']",enggak pakai ovo buat bayar,@KAI121 kenapa gabisa pake ovo buat pembayaran...,@KAI121 kenapa gabisa pake ovo buat pembayaran...,enggak pakai ovo buat bayar,enggak pakai ovo buat bayar
2,1608263340564058116,Thu Dec 29 00:47:15 +0000 2022,1250731212471074816,1608263340564058116,@KAI121 min saya mau ubah no. Hp yang ada di k...,NETRAL,@kai121 min saya mau ubah no. hp yang ada di k...,min saya mau ubah no hp yang ada di kaiaccess ...,min saya mau ubah no hp yang ada di kaiaccess ...,"['min', 'saya', 'mau', 'ubah', 'no', 'hp', 'ya...","['mau', 'ubah', 'no', 'hp', 'bantuannya']","['mau', 'ubah', 'no', 'hp', 'bantu']",mau ubah no hp bantu,@KAI121 min saya mau ubah no. Hp yang ada di k...,@KAI121 min saya mau ubah no. Hp yang ada di k...,mau ubah no hp bantu,mau ubah no hp bantu
3,1608235238366535680,Wed Dec 28 22:55:35 +0000 2022,120352418,1608227080503955458,Waktunya menikmati pemandangan sepanjang perja...,POSITIF,waktunya menikmati pemandangan sepanjang perja...,waktunya menikmati pemandangan sepanjang perja...,waktunya menikmati pemandangan sepanjang perja...,"['waktunya', 'menikmati', 'pemandangan', 'sepa...","['waktunya', 'menikmati', 'pemandangan', 'sepa...","['waktu', 'nikmat', 'pandang', 'panjang', 'jal...",waktu nikmat pandang panjang jalan tani gelas ...,Waktunya menikmati pemandangan sepanjang perja...,Waktunya menikmati pemandangan sepanjang perja...,waktu nikmat pandang panjang jalan tani gelas ...,waktu nikmat pandang panjang jalan tani gelas ...
4,1608049638111850496,Wed Dec 28 10:38:04 +0000 2022,967360502652223489,1608049638111850496,@KAI121 kenapa KAIAccess dari kemaren susah di...,NEGATIF,@kai121 kenapa kaiaccess dari kemaren susah di...,kenapa kaiaccess dari kemaren susah di acces l...,kenapa kaiaccess dari kemarin susah di acces l...,"['kenapa', 'kaiaccess', 'dari', 'kemarin', 'su...","['kemarin', 'susah', 'acces', 'loading', 'terus']","['kemarin', 'susah', 'acces', 'loading', 'terus']",kemarin susah acces loading terus,@KAI121 kenapa KAIAccess dari kemaren susah di...,@KAI121 kenapa KAIAccess dari kemaren susah di...,kemarin susah acces loading terus,kemarin susah acces loading terus
5,1607954768332492802,Wed Dec 28 04:21:05 +0000 2022,1010877687932440577,1607954768332492802,@KAI121 halo saya abis beli tiket kereta di ap...,NEGATIF,@kai121 halo saya abis beli tiket kereta di ap...,halo saya abis beli tiket kereta di aplikasi k...,halo saya habis beli tiket kereta di aplikasi ...,"['halo', 'saya', 'habis', 'beli', 'tiket', 'ke...","['halo', 'habis', 'beli', 'tiket', 'aplikasi',...","['halo', 'habis', 'beli', 'tiket', 'aplikasi',...",halo habis beli tik

In [13]:
from sklearn.model_selection import train_test_split

X = df["text_stemmed"].astype(str).tolist()  # pilih kolom teks yang mau dipakai
y = df["label"].astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))


Train size: 1147
Test size: 287


In [14]:
import torch
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "indolem/indobertweet-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

def get_embeddings(texts, tokenizer, model, max_len=128):
    tokens = tokenizer(
        texts,
        max_length=max_len,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state[:,0,:].numpy()  # CLS token

train_embeds = get_embeddings(X_train, tokenizer, model)
test_embeds = get_embeddings(X_test, tokenizer, model)

train_embeds.shape, test_embeds.shape

((1147, 768), (287, 768))

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

params = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20]
}

rf = RandomForestClassifier(random_state=42, class_weight="balanced")
grid = GridSearchCV(rf, params, cv=3, scoring="f1_macro", n_jobs=-1)
grid.fit(train_embeds, y_train)

print("Best Params:", grid.best_params_)

y_pred = grid.best_estimator_.predict(test_embeds)
print(classification_report(y_test, y_pred, digits=4))

Best Params: {'max_depth': 10, 'n_estimators': 200}
              precision    recall  f1-score   support

     NEGATIF     0.7160    0.4957    0.5859       117
      NETRAL     0.6108    0.8611    0.7147       144
     POSITIF     1.0000    0.1154    0.2069        26

    accuracy                         0.6446       287
   macro avg     0.7756    0.4907    0.5025       287
weighted avg     0.6890    0.6446    0.6162       287

